In [70]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch
import os
import json
from PIL import Image
import pandas as pd
device = "cuda" if torch.cuda.is_available() else "cpu"

In [19]:
def modelPreparation(modelCheckpoint):    
    processor = AutoProcessor.from_pretrained(modelCheckpoint)
    model = Blip2ForConditionalGeneration.from_pretrained(modelCheckpoint,torch_dtype=torch.float16)
    return  model.to(device),processor

In [20]:
modelCheckpoint = "Salesforce/blip2-opt-2.7b"
model,processor = modelPreparation(modelCheckpoint)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
def showimage(image,show=False):
    image = Image.open(image).convert('RGB')
    if show:
        display(image.resize((596,437)))
    return image
    

def normalCaptioning(model,processor,image):
    image = showimage(image)
    inputs = processor(image,return_tensors="pt").to(device,torch.float16)
    ids = model.generate(**inputs,max_new_tokens=50)
    caption = processor.batch_decode(ids,skip_special_tokens=True)[0].strip()
    return caption


a dog is sleeping in a basket of shoes


In [31]:
def promptedCaptioning(model,processor,prompt,image):
    image= showimage(image)
    inputs = processor(image,text=prompt,return_tensors="pt").to(device, torch.float16)
    ids = model.generate(**inputs,max_new_tokens=100)
    caption = processor.batch_decode(ids,skip_special_tokens=True)[0].strip()
    return caption
    

In [57]:
image = '../data/val2014/COCO_val2014_000000546095.jpg'
# caption = normalCaptioning(model,processor,image)
# print(caption)
prompt = 'image of'
caption = promptedCaptioning(model,processor,prompt,image)
print(caption)

two horses grazing in a field


In [79]:
d = {"id":[],"caption":[],"question":[],"gold_answer":[],"predicted_answer":[]}
with open('../data/blip2_vqav2_images_data_sampled.json', 'r') as fcc:
    data = json.load(fcc)
    print(data['546095005'].keys())
    for k in data.keys():
        q = data[k]['question']
        image = '../data/val2014/'+data[k]['img'].split('/')[-1]
        prompt =q+' ,Answer:'
        predAns = promptedCaptioning(model,processor,prompt,image)
        d["id"].append(k)
        d["caption"].append(data[k]['caption'])
        d["question"].append(q)
        d["gold_answer"].append(data[k]['label_answer'])
        d["predicted_answer"].append(predAns)
       
        
    df = pd.DataFrame(d).reset_index() 
    print(df[["gold_answer","predicted_answer"]])
        
    

dict_keys(['img', 'caption', 'question', 'label_answer'])
   gold_answer            predicted_answer
0   dandelions                     daisies
1      nothing                  A suitcase
2           no                         Yes
3           no  Yes, they are in a bathtub
4           no          No, this is a ship
5        white                       white
6           no                         yes
7          yes                         Yes
8            2                           1
9           no                         yes
10      turkey                    a turkey
11        male                        Male
12         yes                         Yes
13         yes                         Yes
14         yes                         Yes
15       black                       green
16         cat                     the cat
17     chicken      Broccoli and mushrooms
18      skiing                      Skiing
19        gray                       Black
